In [ ]:
# Import libs
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd
import math

# Part I

## 1.1. Linear Regression

### Exercise 1

In [ ]:
def polynomial_matrix(x, degree):
  """Compute the polynomial basis matrix of the given degree for vector x"""
  result = np.empty((x.shape[0], degree))
  for k in range(0, degree): # Adds bias
    result[:,k] = x**k
  return result

def fit_polynomial(x, y, degree):
  """Fit a polynomial basis of the given degree to a data set"""
  X = polynomial_matrix(x, degree)
  weights = np.linalg.pinv(X) @ y
  return weights;

# Data set
x = np.array([1,2,3,4])
y = np.array([3,2,0,5])

# 1a
for k in range(1, 5):
  # Fit a polynomial of dimension k to the data set
  weights = fit_polynomial(x, y, k)
  curve_x = np.arange(0,5,0.01)
  curve_y = np.matmul(polynomial_matrix(curve_x, k), weights)
  # Plot the fitted polynomial
  plt.plot(curve_x, curve_y, label=f'k={k}')

# Plot the data set
plt.scatter(x, y)

plt.ylim((-4, 8))
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.savefig(f"q1a", dpi=300)
plt.show()

# 1b
for k in range(1, 5):
  # Print the weights for each fitted polynomial
  weights = fit_polynomial(x, y, k)
  print(f'The weights for k={k} are: {weights}')

# 1c
for k in range(1, 5):
  # Compute MSE on data set
  weights = fit_polynomial(x, y, k)
  X = polynomial_matrix(x, k)
  sse = np.sum((np.matmul(X, weights) - y)**2)
  mse = sse/4
  print(f'The MSE for k={k} is: {mse}')

### Exercise 2

In [ ]:
# 2a i
def f(x):
  return np.sin(2*np.pi*x)**2

def g(sigma, x):
  return f(x) + np.random.normal(0, sigma, x.shape[0])

# Generate training set
size_of_training_set = 30

training_x = np.random.random(size_of_training_set)
training_y = g(0.07, training_x)

# Plot the function without random noise
curve_x = np.arange(0,1,0.01)
curve_y = f(curve_x)

plt.scatter(training_x, training_y)
plt.plot(curve_x, curve_y)

plt.xlabel('x')
plt.ylabel('y')
plt.savefig(f"q2ai", dpi=300)
plt.show()

In [ ]:
# 2a ii
curve_x = np.arange(0,1,0.01)

for k in (2, 5, 10, 14, 18):
  curve_y = np.matmul(polynomial_matrix(curve_x, k), fit_polynomial(training_x, training_y, k))
  plt.plot(curve_x, curve_y, label=f"k={k}", c='orange')

  plt.scatter(training_x, training_y)

  plt.ylim((0, 1.3))

  plt.xlabel('x')
  plt.ylabel('y')
  plt.legend()
  plt.savefig(f"q2aii_{k}", dpi=300)
  plt.show()

In [ ]:
# 2b
dimensions = range(1, 18+1)
mean_squared_errors = []

for k in dimensions:
  # Compute MSE on training set
  weights = fit_polynomial(training_x, training_y, k)
  X = polynomial_matrix(training_x, k)
  sse = np.sum((training_y - np.matmul(X, weights)**2))
  mean_squared_errors.append(sse/size_of_training_set)

plt.plot(dimensions, np.log(mean_squared_errors), label='Training set')
plt.xlabel('Polynomial dimension')
plt.xticks(np.arange(0, 19, step = 2), np.arange(0, 19, step = 2))
plt.ylabel('Mean squared error (ln)')
plt.legend()
plt.savefig(f"q2b", dpi=300)
plt.show()

In [ ]:
# 2c
# Generate test set
size_of_test_set = 1000

test_x = np.random.random(size_of_test_set)
test_y = g(0.07, test_x)

dimensions = range(1, 18+1)
mean_squared_errors = []

for k in dimensions:
  # Compute MSE on test set
  weights = fit_polynomial(training_x, training_y, k)
  X = polynomial_matrix(test_x, k)
  sse = np.sum((test_y - np.matmul(X, weights))**2)
  mean_squared_errors.append(sse/size_of_test_set)

plt.plot(dimensions, np.log(mean_squared_errors), label='Test set')
plt.xlabel('Polynomial dimension')
plt.xticks(np.arange(0, 19, step = 2), np.arange(0, 19, step = 2))
plt.ylabel('Mean squared error (ln)')
plt.legend()
plt.savefig(f"q2c", dpi=300)
plt.show()

In [ ]:
# 2d
# Repeat (b) and (c) for 100 runs
number_of_runs = 100
dimensions = range(1, 18+1)

set_of_training_errors = []
set_of_test_errors = []

for i in range(number_of_runs):
  # Generate training set
  training_x = np.random.random(size_of_training_set)
  training_y = g(0.07, training_x)

  training_mean_squared_errors = []

  for k in dimensions:
    # Compute MSE on training set
    weights = fit_polynomial(training_x, training_y, k)
    X = polynomial_matrix(training_x, k)
    sse = np.sum((training_y - np.matmul(X, weights))**2)
    training_mean_squared_errors.append(sse/size_of_training_set)
  
  # Generate training set
  test_x = np.random.random(size_of_test_set)
  test_y = g(0.07, test_x)

  test_mean_squared_errors = []

  for k in dimensions:
    # Compute MSE on test set
    weights = fit_polynomial(training_x, training_y, k)
    X = polynomial_matrix(test_x, k)
    sse = np.sum((test_y - np.matmul(X, weights))**2)
    test_mean_squared_errors.append(sse/size_of_test_set)

  set_of_training_errors.append(training_mean_squared_errors)
  set_of_test_errors.append(test_mean_squared_errors)

set_of_training_errors = np.array(set_of_training_errors)
set_of_test_errors = np.array(set_of_test_errors)

# Average the training and test errors for each dimension k
average_training_errors = set_of_training_errors.mean(axis=0)
average_test_errors = set_of_test_errors.mean(axis=0)

# Plot the natural log of the average training and test errors
plt.plot(dimensions, np.log(average_training_errors), label='Training data')
plt.plot(dimensions, np.log(average_test_errors), label='Test data')

plt.xlabel('Polynomial dimension')
plt.xticks(np.arange(0, 19, step = 2), np.arange(0, 19, step = 2))
plt.ylabel('Average mean squared error (ln)')
plt.legend()
plt.savefig(f"q2d", dpi=300)
plt.show()

### Exercise 3

In [ ]:
def basis_matrix(x, degree):
  """Compute the basis matrix of the given degree for vector x"""
  result = np.empty((x.shape[0], degree))
  for k in range(1, degree+1):
    result[:,k-1] = np.sin(k*np.pi*x)
  return result

def fit_basis(x, y, degree):
  """Fit a basis of the given degree to a data set"""
  X = basis_matrix(x, degree)
  weights = np.matmul(np.linalg.pinv(X), y)
  return weights;

dimensions = range(1, 18+1)
size_of_training_set = 30
size_of_test_set = 1000

# Repeat of 2b
mean_squared_errors = []

# Generate training set
training_x = np.random.random(size_of_training_set)
training_y = g(0.07, training_x)

for k in dimensions:
  # Calculate MSE on training set
  weights = fit_basis(training_x, training_y, k)
  X = basis_matrix(training_x, k)
  sse = np.sum((f(training_x) - np.matmul(X, weights))**2)
  mean_squared_errors.append(sse/size_of_training_set)

plt.plot(dimensions, np.log(mean_squared_errors), label='Training set')
plt.xlabel('Basis dimension')
plt.ylabel('Mean squared error (ln)')
plt.legend()
plt.savefig(f"q3_2b", dpi=300)
plt.show()

# Repeat of 2c
mean_squared_errors = []

# Generate test set
test_x = np.random.random(size_of_test_set)
test_y = g(0.07, test_x)

for k in dimensions:
  # Compute MSE on test set
  weights = fit_basis(training_x, training_y, k)
  X = basis_matrix(test_x, k)
  sse = np.sum((test_y - np.matmul(X, weights))**2)
  mean_squared_errors.append(sse/size_of_test_set)

plt.plot(dimensions, np.log(mean_squared_errors), label='Test set')
plt.xlabel('Basis dimension')
plt.ylabel('Mean squared error (ln)')
plt.legend()
plt.savefig(f"q3_2c", dpi=300)
plt.show()

# Repeat of 2d
number_of_runs = 100

set_of_training_errors = []
set_of_test_errors = []

for i in range(number_of_runs):
  # Generate training set
  training_x = np.random.random(size_of_training_set)
  training_y = g(0.07, training_x)

  training_mean_squared_errors = []

  for k in dimensions:
    # Compute MSE on training set
    weights = fit_basis(training_x, training_y, k)
    X = basis_matrix(training_x, k)
    sse = np.sum((training_y - np.matmul(X, weights))**2)
    training_mean_squared_errors.append(sse/size_of_training_set)
  
  # Generate test set
  test_x = np.random.random(size_of_test_set)
  test_y = g(0.07, test_x)

  test_mean_squared_errors = []

  for k in dimensions:
    # Compute MSE on test set
    weights = fit_basis(training_x, training_y, k)
    X = basis_matrix(test_x, k)
    sse = np.sum((test_y - np.matmul(X, weights))**2)
    test_mean_squared_errors.append(sse/size_of_test_set)

  set_of_training_errors.append(training_mean_squared_errors)
  set_of_test_errors.append(test_mean_squared_errors)

set_of_training_errors = np.array(set_of_training_errors)
set_of_test_errors = np.array(set_of_test_errors)

# Average the training and test errors for each dimension k
average_training_errors = set_of_training_errors.mean(axis=0)
average_test_errors = set_of_test_errors.mean(axis=0)

# Plot the natural log of the training and test errors
plt.plot(dimensions, np.log(average_training_errors), label='Training data')
plt.plot(dimensions, np.log(average_test_errors), label='Test data')

plt.xlabel('Basis dimension')
plt.ylabel('Average mean squared error (ln)')
plt.legend()
plt.savefig(f"q3_2d", dpi=300)
plt.show()

## 1.2: Filtered Boston housing and kernels

In [ ]:
# Import the data as 'housedata_df'
housedata_df = pd.read_csv('http://www0.cs.ucl.ac.uk/staff/M.Herbster/boston-filter/Boston-filtered.csv')
housedata_df

In [ ]:
housedata = housedata_df.to_numpy()

def generate_samples(data, frac=2/3):
  """General function to generate training and test sets, as well as arrays of output values for each set"""
  n = data.shape[0]
  shuffled_indices = [*range(n)]
  np.random.shuffle(shuffled_indices)
  # Generate random training and test sets
  training_n = int(frac * n)
  training_indices, test_indices = shuffled_indices[:training_n], shuffled_indices[training_n:]
  training_set, test_set = data[training_indices], data[test_indices]
  # Extract relevant output variable (13th column) of training and test sets
  training_price = training_set[:,12]
  test_price = test_set[:,12]
  return training_set, test_set, training_price, test_price

### Exercise 4

#### (a) Naive regression

In [ ]:
mse_training = []
mse_test = []

for i in range(20):
  # Generate training and test sets, as well as arrays of output variable for each set
  training_set_boston, test_set_boston, training_price_array, test_price_array = generate_samples(housedata)

  # Create vectors of ones that are the same length as the training and testing sets, respectively
  ones_training_set = np.ones(training_set_boston.shape[0])
  ones_test_set = np.ones(test_set_boston.shape[0])

  # Calculate single regressor parameter based on training set 
  regressor = (np.matmul(ones_training_set.T,ones_training_set))**(-1)*(np.matmul(ones_training_set.T,training_price_array))
  
  # Compute MSE on training set
  mse_training_single_run = np.sum((training_price_array - (regressor*ones_training_set))**2)/training_set_boston.shape[0]
  mse_training.append(mse_training_single_run)

  # Compute MSE on test set
  mse_test_single_run = np.sum((test_price_array - (regressor*ones_test_set))**2)/test_set_boston.shape[0]
  mse_test.append(mse_test_single_run)

# Print mean of MSE over 20 runs
print(f'MSE training error over 20 runs based on naive regression: {round(sum(mse_training)/20, 2)}')
print(f'MSE test error over 20 runs based on naive regression: {round(sum(mse_test)/20, 2)}')

We obtain the following result:

*   MSE training error over 20 runs based on naive regression: 83.01
*   MSE test error over 20 runs based on naive regression: 87.46

#### (b) Interpretation of constant function

The constant function is equivalent to the *mean value of column 13 for the training set*.

This can be derived from the expression used to determine the regression parameter: the first factor ($X^TX$) is the inverse of the number of observations in the training set, whereas the second term ($X^TY$) is the sum of the values of column 13 (i.e. median house price) in the training set. In other words, the constant function is the mean value of the feature in question.

#### (c) Linear Regression with single attributes

In [ ]:
for k in range(0,12):

  mse_training = []
  mse_test = []
  for i in range(20):
  # Generate training and test sets, as well as arrays of output variable for each set
    training_set_boston, test_set_boston, training_price_array, test_price_array = generate_samples(housedata)

  # Extract relevant input variable of training and test sets, and generate arrays
    training_feature_array = training_set_boston[:,k]
    test_feature_array = test_set_boston[:,k]
  
  # Add bias
    ones_training_set = np.ones(training_set_boston.shape[0])
    ones_test_set = np.ones(test_set_boston.shape[0])
    training_matrix_single = np.stack((training_feature_array, ones_training_set), axis = 1)
    test_matrix_single = np.stack((test_feature_array, ones_test_set), axis = 1)

  # Calculate regressor vector based on training set 
    regressor_single = np.matmul((np.linalg.inv(np.matmul(training_matrix_single.T,training_matrix_single))),(np.matmul(training_matrix_single.T,training_price_array)))

  # Compute MSE on training set
    mse_training_single_run = np.sum((training_price_array - (np.matmul(training_matrix_single, regressor_single)))**2)/training_set_boston.shape[0]
    mse_training.append(mse_training_single_run)

  # Compute MSE on test set
    mse_test_single_run = np.sum((test_price_array - (np.matmul(test_matrix_single, regressor_single)))**2)/test_set_boston.shape[0]
    mse_test.append(mse_test_single_run)

  print(f'Attribute {k+1} ({housedata_df.columns[k].strip()})')
  print(f'Training MSE over 20 runs: {round(sum(mse_training)/20, 2)}')
  print(f'Test MSE over 20 runs: {round(sum(mse_test)/20, 2)}')
  print()

We obtain the following results:

1.   Attribute 1 (CRIM)

  *   Training MSE over 20 runs: 69.30
  *   Test MSE over 20 runs: 77.31

2.   Attribute 2 (ZN)

  *   Training MSE over 20 runs: 71.50
  *   Test MSE over 20 runs: 77.79

3.   Attribute 3 (INDUS)

  *   Training MSE over 20 runs: 65.96
  *   Test MSE over 20 runs: 62.47

4.   Attribute 4 (CHAS)
  *   Training MSE over 20 runs: 81.06
  *   Test MSE over 20 runs: 83.75

5.   Attribute 5 (NOX)
  *   Training MSE over 20 runs: 70.83
  *   Test MSE over 20 runs: 65.61

6.   Attribute 6 (RM)
  *   Training MSE over 20 runs: 44.81
  *   Test MSE over 20 runs: 41.55

7.   Attribute 7 (AGE)
  *   Training MSE over 20 runs: 69.09
  *   Test MSE over 20 runs: 79.59

8.   Attribute 8 (DIS)
  *   Training MSE over 20 runs: 80.00
  *   Test MSE over 20 runs: 77.85

9.   Attribute 9 (RAD)
  *   Training MSE over 20 runs: 73.66
  *   Test MSE over 20 runs: 69.44

10.   Attribute 10 (TAX)
  *   Training MSE over 20 runs: 66.08
  *   Test MSE over 20 runs: 65.86

11.   Attribute 11 (PTRATIO)
  *   Training MSE over 20 runs: 63.38
  *   Test MSE over 20 runs: 61.65

12.   Attribute 12 (LSTAT)
  *   Training MSE over 20 runs: 38.62
  *   Test MSE over 20 runs: 38.59

Taken in isolation, Attributes 6 and 12 provide the most accurate prediction for the value of the 13th column.

#### (d) Linear Regression using all attributes

In [ ]:
  mse_training = []
  mse_test = []

  for i in range(20):
    # Generate training and test sets, as well as arrays of output variable for each set
    training_set_boston, test_set_boston, training_price_array, test_price_array = generate_samples(housedata)

    # Extract relevant input variables of training and test sets, and generate arrays
    training_features_all = training_set_boston[:,:-1]
    test_features_all = test_set_boston[:,:-1]
  
    # Add bias
    ones_training_set = np.ones((training_set_boston.shape[0], 1))
    ones_test_set = np.ones((test_set_boston.shape[0], 1))
    training_matrix_all = np.hstack((training_features_all, ones_training_set))
    test_matrix_all = np.hstack((test_features_all, ones_test_set))

    # Calculate regressor vector based on training set 
    regressor_all = np.matmul((np.linalg.inv(np.matmul(training_matrix_all.T,training_matrix_all))),(np.matmul(training_matrix_all.T,training_price_array)))

    # Compute MSE on training set
    mse_training_single_run = np.sum((training_price_array - (np.matmul(training_matrix_all, regressor_all)))**2)/training_set_boston.shape[0]
    mse_training.append(mse_training_single_run)

    # Compute MSE on test set
    mse_test_single_run = np.sum((test_price_array - (np.matmul(test_matrix_all, regressor_all)))**2)/test_set_boston.shape[0]
    mse_test.append(mse_test_single_run)

  print(f'Training MSE over 20 runs: {round(sum(mse_training)/20, 2)}')
  print(f'Test MSE over 20 runs: {round(sum(mse_test)/20, 2)}')

We obtain the following results:

*   Training MSE over 20 runs: 21.98
*   Test MSE over 20 runs: 24.99

The MSEs for both the training and test sets are lower than for any of the models based on a single feature.

### Exercise 5



#### (a) Cross-validation

In [ ]:
# Generate training and test sets, as well as arrays of output variable for each set
training_set_boston, test_set_boston, training_price_array, test_price_array = generate_samples(housedata)

In [ ]:
def gen_validation_folds(data, k):
  """Split data set into k parts. Returns a list of k lists of the forms
  [training_set, validation_set], where the validation_set is a fraction (1/k)
  of data, and the training_set is data without this chunk"""
  result = []

  n = data.shape[0]
  shuffled_indices = np.arange(n)
  np.random.shuffle(shuffled_indices)
  # Split the shuffled indices into k chunks for the validation sets
  for validation_indices in np.array_split(shuffled_indices, k):
      training_indices = np.setdiff1d(shuffled_indices, validation_indices)
      # Extract training and validation sets
      training_set, validation_set = data[training_indices], data[validation_indices]
      result.append([training_set, validation_set])
  return result

validation_folds = gen_validation_folds(training_set_boston, 5)

In [ ]:
# Define kernel function
def kernel(a, b, sigma):
  """Compute Gaussian kernel of a and b with parameter sigma"""
  result = np.exp(-(np.linalg.norm(a-b))**2/(2*(sigma**2)))
  return result

# Create a vector of gamma values
gamma_values = []
gamma_powers = []
for i in reversed(range(26, 40+1, 1)):
  gamma_values.append(2**(-i))
  gamma_powers.append(i)

# Create a vector of sigma values
sigma_values = []
sigma_powers = []
for i in np.arange(7, 13.5, 0.5):
  sigma_values.append(2**i)
  sigma_powers.append(i)

In [ ]:
import time
start_time = time.time()

error_matrix = np.zeros((len(gamma_values), len(sigma_values)))
for g in range(len(gamma_values)):
  gamma = gamma_values[g]
  gamma_power = gamma_powers[g]
  for s in range(len(sigma_values)):
    sigma = sigma_values[s]
    sigma_power = sigma_powers[s]
    error = []
    print("Computing: gamma_values[%s] --- sigma_values[%s] --- %s seconds" % (g, s, (time.time() - start_time)))
    for f in range(5):
      training_fold = validation_folds[f][0]
      validation_fold = validation_folds[f][1]

      # Compute Kernel matrix
      K_matrix = np.zeros((training_fold.shape[0], training_fold.shape[0]))
      for i in range(training_fold.shape[0]):
        for j in range(training_fold.shape[0]):
          K_matrix[i,j] = kernel(training_fold[i,:-1], training_fold[j,:-1], sigma)
      # Compute alpha vector
      identity_matrix = np.identity(training_fold.shape[0])
      alpha = np.matmul((np.linalg.inv(K_matrix + (gamma * training_fold.shape[0] * identity_matrix))), training_fold[:,-1])
      # Compute error for every data point within the given fold
      for i in range(validation_fold.shape[0]):
        individual_terms = []
        for j in range(training_fold.shape[0]):
          individual_terms.append(alpha[j]*kernel(training_fold[j,:-1], validation_fold[i,:-1], sigma))
        predicted_value = (sum(individual_terms))
        individual_error = abs(predicted_value - validation_fold[i,-1])
        error.append(individual_error)
    # Compute average error for a given pairing of gamma and sigma
    average_error = sum(error)/training_set_boston.shape[0]
    error_matrix[g,s] = average_error

error_df = pd.DataFrame(data=error_matrix, index=gamma_powers[:], columns=sigma_powers[:])
error_df.to_csv('error_df_2.csv', sep=',')

In [ ]:
# Determine minimum value of the mean error
minimum = np.amin(error_matrix)

# Identify the values of sigma and gamma that generated the minimum value:
for i in range(error_matrix.shape[0]):
  for j in range(error_matrix.shape[1]):
    if error_matrix[i,j] == minimum:
      best_gamma = gamma_values[i]
      best_sigma = sigma_values[j]
      print(f'Minimum mean error is achieved with gamma = 2^-{gamma_powers[i]}, and sigma = 2^{sigma_powers[j]}')

In [ ]:
minimum

In [ ]:
error_matrix

#### (b) Plotting the cross-validation error

In [ ]:
cross_validation_error = error_matrix/5

cross_validation_error

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
X = np.array(sigma_powers)
Y = np.array(gamma_powers)
Z = cross_validation_error

X_mesh, Y_mesh = np.meshgrid(X, Y)
ax.set_xlabel('Sigma')
ax.set_ylabel('Gamma')
ax.set_zlabel('Cross-validation error')
ax.dist = 11 # Avoid Z-axis label getting cut off
ax.scatter(X_mesh, Y_mesh, Z)
plt.show()
fig.savefig(f"q5b", dpi=300)

#### (c) MSE for best values of $\gamma$ and $\sigma$

In [ ]:
gamma = best_gamma
sigma = best_sigma
# Compute Kernel matrix
K_matrix = np.zeros((training_set_boston.shape[0], training_set_boston.shape[0]))
for i in range(training_set_boston.shape[0]):
    for j in range(training_set_boston.shape[0]):
        K_matrix[i,j] = kernel(training_set_boston[i,:-1], training_set_boston[j,:-1], sigma)
# Compute alpha vector
identity_matrix = np.identity(training_set_boston.shape[0])
alpha = np.matmul((np.linalg.inv(K_matrix + (gamma * training_set_boston.shape[0] * identity_matrix))), training_set_boston[:,-1])
# Compute square error for data points from training set
error_training = []
for i in range(training_set_boston.shape[0]):
  individual_terms = []
  for j in range(training_set_boston.shape[0]):
    individual_terms.append(alpha[j]*kernel(training_set_boston[j,:-1], training_set_boston[i,:-1], sigma))
  predicted_value = (sum(individual_terms))
  individual_square_error = (predicted_value - training_set_boston[i,-1])**2
  error_training.append(individual_square_error)
# Compute square error for data points from test set
error_test=[]
for i in range(test_set_boston.shape[0]):
  individual_terms = []
  for j in range(training_set_boston.shape[0]):
    individual_terms.append(alpha[j]*kernel(training_set_boston[j,:-1], test_set_boston[i,:-1], sigma))
  predicted_value = (sum(individual_terms))
  individual_square_error = (predicted_value - test_set_boston[i,-1])**2
  error_test.append(individual_square_error)
# Compute mean square error
average_error_training = sum(error_training)/training_set_boston.shape[0]
print(f'MSE on training set is {round(average_error_training, 2)}')
average_error_test = sum(error_test)/test_set_boston.shape[0]
print(f'MSE on test set is {round(average_error_test, 2)}')

we obtain the following results:
* MSE on training set is approx. 7.12
* MSE on test set is approx. 12.84

#### (d) Comparison of all methods

In [ ]:
runs = []
for i in range(1,20+1):
  runs.append(f'Run {i}')

In [ ]:
methods = ['Naive Regression']
for i in range(1, 12+1):
  methods.append(f'Linear regression (attribute {i})')
methods.extend(('Linear regression (all attributes)', 'Kernel Ridge Regression'))

In [ ]:
train_MSE = pd.DataFrame(index=methods, columns=runs)
test_MSE = pd.DataFrame(index=methods, columns=runs)

In [ ]:
for r in range(20):
  # Generate training and test sets, as well as arrays of output variable for each set
  training_set_boston, test_set_boston, training_price_array, test_price_array = generate_samples(housedata)

# Naive regression
  # Create vectors of ones that are the same length as the training and testing sets, respectively
  ones_training_set = np.ones(training_set_boston.shape[0])
  ones_test_set = np.ones(test_set_boston.shape[0])

  # Calculate single regressor parameter based on training set 
  regressor = (np.matmul(ones_training_set.T,ones_training_set))**(-1)*(np.matmul(ones_training_set.T,training_price_array))
  
  # Compute MSE on training set
  mse_training_single_run = np.sum((training_price_array - (regressor*ones_training_set))**2)/training_set_boston.shape[0]
  train_MSE.iloc[0,r] = mse_training_single_run

  # Compute MSE on test set
  mse_test_single_run = np.sum((test_price_array - (regressor*ones_test_set))**2)/test_set_boston.shape[0]
  test_MSE.iloc[0,r] = mse_test_single_run

# Linear regression based on single attribute
  for k in range(0,12):

  # Extract relevant input variable of training and test sets, and generate arrays
    training_feature_array = training_set_boston[:,k]
    test_feature_array = test_set_boston[:,k]
  
  # Add bias
    ones_training_set = np.ones(training_set_boston.shape[0])
    ones_test_set = np.ones(test_set_boston.shape[0])
    training_matrix_single = np.stack((training_feature_array, ones_training_set), axis = 1)
    test_matrix_single = np.stack((test_feature_array, ones_test_set), axis = 1)

  # Calculate regressor vector based on training set 
    regressor_single = np.matmul((np.linalg.inv(np.matmul(training_matrix_single.T,training_matrix_single))),(np.matmul(training_matrix_single.T,training_price_array)))

  # Compute MSE on training set
    mse_training_single_run = np.sum((training_price_array - (np.matmul(training_matrix_single, regressor_single)))**2)/training_set_boston.shape[0]
    train_MSE.iloc[k+1,r] = mse_training_single_run

  # Compute MSE on test set
    mse_test_single_run = np.sum((test_price_array - (np.matmul(test_matrix_single, regressor_single)))**2)/test_set_boston.shape[0]
    test_MSE.iloc[k+1,r] = mse_test_single_run
  
# Linear regression based on all attributes
  # Extract relevant input variables of training and test sets, and generate arrays
  training_features_all = training_set_boston[:,:-1]
  test_features_all = test_set_boston[:,:-1]
  
  # Add bias
  ones_training_set = np.ones((training_set_boston.shape[0], 1))
  ones_test_set = np.ones((test_set_boston.shape[0], 1))
  training_matrix_all = np.hstack((training_features_all, ones_training_set))
  test_matrix_all = np.hstack((test_features_all, ones_test_set))

  # Calculate regressor vector based on training set 
  regressor_all = np.matmul((np.linalg.inv(np.matmul(training_matrix_all.T,training_matrix_all))),(np.matmul(training_matrix_all.T,training_price_array)))

  # Compute MSE on training set
  mse_training_single_run = np.sum((training_price_array - (np.matmul(training_matrix_all, regressor_all)))**2)/training_set_boston.shape[0]
  train_MSE.iloc[13,r] = mse_training_single_run

  # Compute MSE on test set
  mse_test_single_run = np.sum((test_price_array - (np.matmul(test_matrix_all, regressor_all)))**2)/test_set_boston.shape[0]
  test_MSE.iloc[13,r] = mse_test_single_run

# Kernel Ridge Regression
  
  # Step 1: Identify best values of sigma and gamma 
  # Split training set into validation folds
  validation_folds = gen_validation_folds(training_set_boston, 5)

  error_matrix = np.zeros((len(gamma_values), len(sigma_values)))
  for g in range(len(gamma_values)):
    gamma = gamma_values[g]
    gamma_power = gamma_powers[g]
    for s in range(len(sigma_values)):
      sigma = sigma_values[s]
      sigma_power = sigma_powers[s]
      error = []
      for f in range(5):
        training_fold = validation_folds[f][0]
        validation_fold = validation_folds[f][1]
        # Compute Kernel matrix
        K_matrix = np.zeros((training_fold.shape[0], training_fold.shape[0]))
        for i in range(training_fold.shape[0]):
          for j in range(training_fold.shape[0]):
            K_matrix[i,j] = kernel(training_fold[i,:-1], training_fold[j,:-1], sigma)
        # Compute alpha vector
        identity_matrix = np.identity(training_fold.shape[0])
        alpha = np.matmul((np.linalg.inv(K_matrix + (gamma * training_fold.shape[0] * identity_matrix))), training_fold[:,-1])
        # Compute error for every data point within the given fold
        for i in range(validation_fold.shape[0]):
          individual_terms = []
          for j in range(training_fold.shape[0]):
            individual_terms.append(alpha[j]*kernel(training_fold[j,:-1], validation_fold[i,:-1], sigma))
          predicted_value = (sum(individual_terms))
          individual_error = abs(predicted_value - validation_fold[i,-1])
          error.append(individual_error)
      # Compute average error for a given pairing of gamma and sigma
      average_error = sum(error)/training_set_boston.shape[0]
      error_matrix[g,s] = average_error
  
  # Determine minimum value of the mean error
  minimum = np.amin(error_matrix)

  # Identify the values of sigma and gamma that generated the minimum value:
  for i in range(error_matrix.shape[0]):
    for j in range(error_matrix.shape[1]):
      if error_matrix[i,j] == minimum:
        best_gamma = gamma_values[i]
        best_sigma = sigma_values[j]

  # Step 2: Apply Kernel Ridge Regression on training and test set using best values 
  gamma = best_gamma
  sigma = best_sigma

  # Compute Kernel matrix
  K_matrix = np.zeros((training_set_boston.shape[0], training_set_boston.shape[0]))
  for i in range(training_set_boston.shape[0]):
      for j in range(training_set_boston.shape[0]):
          K_matrix[i,j] = kernel(training_set_boston[i,:-1], training_set_boston[j,:-1], sigma)
  # Compute alpha vector
  identity_matrix = np.identity(training_set_boston.shape[0])
  alpha = np.matmul((np.linalg.inv(K_matrix + (gamma * training_set_boston.shape[0] * identity_matrix))), training_set_boston[:,-1])
  # Compute square error for data points from training set
  error_training = []
  for i in range(training_set_boston.shape[0]):
    individual_terms = []
    for j in range(training_set_boston.shape[0]):
      individual_terms.append(alpha[j]*kernel(training_set_boston[j,:-1], training_set_boston[i,:-1], sigma))
    predicted_value = (sum(individual_terms))
    individual_square_error = (predicted_value - training_set_boston[i,-1])**2
    error_training.append(individual_square_error)
  mse_training_single_run = sum(error_training)/training_set_boston.shape[0]
  train_MSE.iloc[14,r] = mse_training_single_run
  # Compute square error for data points from test set
  error_test=[]
  for i in range(test_set_boston.shape[0]):
    individual_terms = []
    for j in range(training_set_boston.shape[0]):
      individual_terms.append(alpha[j]*kernel(training_set_boston[j,:-1], test_set_boston[i,:-1], sigma))
    predicted_value = (sum(individual_terms))
    individual_square_error = (predicted_value - test_set_boston[i,-1])**2
    error_test.append(individual_square_error)
  mse_test_single_run = sum(error_test)/test_set_boston.shape[0]
  test_MSE.iloc[14,r] = mse_test_single_run

train_MSE.to_csv('MSE_train.csv', sep=',')
test_MSE.to_csv('MSE_test.csv', sep=',')

In [ ]:
train_MSE

In [ ]:
summary = pd.DataFrame(index=methods, columns=('MSE train mean', 'MSE train std dev', 'MSE test mean', 'MSE test std dev'))

for i in range(summary.shape[0]):
  summary.iloc[i,0] = train_MSE.iloc[i,:].mean(axis=0)
  summary.iloc[i,1] = train_MSE.iloc[i,:].std(axis=0)
  summary.iloc[i,2] = test_MSE.iloc[i,:].mean(axis=0)
  summary.iloc[i,3] = test_MSE.iloc[i,:].std(axis=0)

In [ ]:
summary.to_csv('summary_ex5.csv', sep=',')

In [ ]:
summary

In [ ]:
test_MSE
